In [3]:
#임포트
import requests
from bs4 import BeautifulSoup as bs
import random
import time
from tqdm import tqdm, trange  # 진행 상황
import os
from urllib.request import urlretrieve, Request, urlopen

- 기존에 있던 250개에 추가
- 텍스트 파일 안 목록 -> beer_list로 가져오기

In [16]:
file=open('./list_crawl2.txt','r', encoding='UTF-8')
All=file.read()
file.close()
beer_list=All.split('\n')

In [17]:
beer_list

['/874/2762/',
 '/83/175423/',
 '/37092/190730/',
 '/16013/92893/',
 '/16013/92928/',
 '/16013/114541/',
 '/2550/7511/',
 '/2970/8759/',
 '/502/1432/',
 '/281/23410/',
 '/367/976/',
 '/779/63536/',
 '/180/1421/',
 '/180/141574/',
 '/579/36594/',
 '/290/6906/',
 '/498/1422/',
 '/4949/45380/',
 '/5142/4299/',
 '/5142/4291/',
 '/1472/229743/',
 '/579/242043/',
 '/14879/132406/',
 '/585/2637/',
 '/16127/73088/',
 '/290/41091/',
 '/12691/27137/',
 '/20914/52126/',
 '/4108/50578/',
 '/613/1641/',
 '/16422/83075/',
 '/360/935/',
 '/437/1317/',
 '/172/115552/',
 '/713/52539/',
 '/7593/62476/',
 '/1332/24720/',
 '/95/280/',
 '/9275/16767/',
 '/212/5798/',
 '/212/26244/',
 '/212/160082/',
 '/813/7451/',
 '/813/134703/',
 '/47/13159/',
 '/34988/129878/',
 '/637/1716/',
 '/180/12738/\u3000',
 '/192/197183/',
 '/192/275731/',
 '/273/751/',
 '/187/600/',
 '/187/601/',
 '/187/6104/',
 '/48/10482/',
 '/25941/100185/',
 '/491/457725/',
 '/140/251571/',
 '/23640/70471/',
 '/3517/20134/',
 '/48/13889/',


In [10]:
#개수 확인
len(beer_list)

4

In [12]:
#페이지 안에서 크롤링
def get_beer_content(url):
    global data    
    base_url = f"http://beeradvocate.com/beer/profile{url}"
    response = requests.get ( base_url )

    soup = bs(response.text, 'lxml')   

    note = soup.select("#ba-content > div:nth-child(8) > div:nth-child(3)")[0].text
    note2 = note.split(":")[-1].strip() 

    titles = soup.find("div", {'class':'titleBar'}).contents[1]    
    title_span = titles.find('span').text    
    title = titles.text.replace(title_span,"")    

    stats = soup.findAll("dd", {'class':'beerstats'})        

    beerData = {"beerNo":url.split("/")[-2], "beerName":title, "type":(stats[0].find('b')).text,
                "country":stats[7].text.split(', ')[-1], "company":stats[6].text,
            "ABV":stats[1].text.split('%')[0], "ratingBA":stats[3].text.split(' ')[0],
            "ratingCnt":stats[5].text.replace(',',''), "Notes":note2}

    return beerData

In [13]:
#크롤링 실행
beer_datas = []
for i, beers in tqdm(enumerate(beer_list)):
    b = get_beer_content(beers)
    beer_datas.append(b)
    time.sleep(random.randint(1, 2))

0it [00:02, ?it/s]


IndexError: list index out of range

In [21]:
#확인
len(beer_datas)

77

In [22]:
beer_datas[76]

{'beerNo': '1944',
 'beerName': 'Spaten Münchner Hell (München/Premium Lager)',
 'type': 'German Helles',
 'country': 'Germany',
 'company': 'Spaten-Franziskaner-Bräu',
 'ABV': ' 5.2',
 'ratingBA': '3.69',
 'ratingCnt': '1577',
 'Notes': 'Marketed as "München" in the UK and "Premium Lager" in the USA.'}

In [32]:
#오라클 연결
import cx_Oracle

connstr = 'bgirls/BGirls@192.168.1.46:1521/orcl'
conn = cx_Oracle.connect(connstr)
curs = conn.cursor()
os.environ["NLS_LANG"] = ".AL32UTF8"

In [33]:
#저장하기
for bd in beer_datas:  
        
    sql = 'INSERT INTO beer(beerNo, beerName, country, company, type, abv, rating, ratingBA, ratingCnt, note) VALUES(:beerNo, :beerName, :country, :company, :type, :abv, :rating, :ratingBA, :ratingCnt, :note)'    
    
    if bd['ABV'] == 'not listed':
        bd['ABV'] = 0
    
    
    try:
        curs.prepare(sql)
        curs.execute(None, beerNo=bd['beerNo'], beerName=bd['beerName'], country=bd['country'], company=bd['company'], type=bd['type'], abv=float(bd['ABV']), rating=0, ratingBA=float(bd['ratingBA']), ratingCnt=int(bd['ratingCnt']), note=bd['Notes'])
        #curs.execute(sql, beerName=bd['beerName'], company=bd['company'], type=bd['type'], abv=float(bd['ABV']), rating=0, ratingBA=float(bd['ratingBA']), note=bd['Notes'])
        conn.commit() 
        
    except Exception as e:
        if str(e).split(':')[0] != 'ORA-00001':            
            print(e, '::', bd['beerName'], bd['beerNo'], bd['company'])
            conn.rollback()
        else:
            conn.rollback()
        
        
curs.close()        
conn.close()


In [28]:
# 사진이 cdn으로 되어있어 따로 따와야함
#사진 저장 메소드
def get_beer_pic(url):
    global data    
    base_url = f"http://beeradvocate.com/beer/profile{url}"
    response = requests.get ( base_url )
    
    if response.status_code == 200:     
        
        soup = bs(response.text, 'lxml')
        pic = soup.findAll('img')[2].get('src')        
        fname = pic.split("/")[-1]
        
        #f = open(fname,'wb')
        #f.write(urllib.request.urlopen(pic).read())
        #f.close()        
        
        f = open(fname, 'wb')
        req = Request(pic, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        f.write(webpage)
        f.close() 
                
        #urlretrieve(pic, fname)
        print("이미지 저장 완료")       
        

In [29]:
# 사진 저장 크롤링. 없는 경우는 가져오지 않음
for i, beer in tqdm(enumerate(beer_list)):
    b = get_beer_pic(beer)    
    time.sleep(random.randint(1, 2))


0it [00:00, ?it/s]

이미지 저장 완료



1it [00:03,  3.84s/it]

이미지 저장 완료



2it [00:08,  4.08s/it]

...


75it [05:08,  3.65s/it]

이미지 저장 완료



76it [05:10,  3.42s/it]

이미지 저장 완료



77it [05:13,  3.24s/it]
78it [05:16,  4.06s/it]
